# Generating Useful Wikidata Files

### Batch Invocation
Example batch command. The second argument is a notebook where the output will be stored. You can load it to see progress.

```
papermill Example7\ -\ Wikidata\ Outputs.ipynb example7.out.ipynb \
-p home /Users/pedroszekely/Downloads/kypher \
-p wiki_file all.10.tsv.gz \
-p output_folder output.all.10 \
-p temp_folder temp.all.10 \
-p delete_database true 
```

In [ ]:
# Parameters
home = "/Users/pedroszekely/Downloads/kypher"
wiki_file = "all.tsv.gz"
wiki_file = "all.10.tsv.gz"
output_folder = "output"
output_folder = "output.all.10"
temp_folder = "temp"
temp_folder = "temp.all.10"
delete_database = "true"

In [2]:
import io
import os
import subprocess
import sys

import numpy as np
import pandas as pd

# from IPython.display import display, HTML, Image
# from pandas_profiling import ProfileReport

## Set up environment and folders to store the files

- `WIKIDATA_HOME` folder where you put your Wikidata data
- `OUT` folder where the output files go
- `TEMP` folder to keep temporary files , including the database
- `kgtk` shortcut to invoke the kgtk software

The current implementation of some of the kgtk commands does not understand compressed files. In particular, `query` often rejects `gz` files.

To dos:

- Make sure that all files have id columns as `query` gets unhappy when files have no ids.
- Create an output folder for a subset of Wikidata without scholarly articles. This is half done: the remaining work is to subtract the scholarly articles from `EDGES` and repeat the workflow.
- Change the naming convention to make it clear which files are a partition of the original `EDGES`, so users know what files they need to get to have a full version.
- Create a qualifier file for the partition files of Wikidata: this is so that if a user gets one of the partitions, they can get the corresponding qualifier file.
- Add pagerank and other stats. We can compute the pagerank from the `all.item` file, so maybe should be called `all.item.pagerank.tsv`

Naming convention: the name `all` is redundant, we should consider removing it. I recomment using the prefix `part.` to name the partition of Wikidata, e.g., `part.label`, `part.quantity`. Files such as `P279` are not partitions as it is a subset of `part.item`.

If we create a subset of Wikidata, e.g., no scholarly articles, we could call it `minus.Q13442814`; if we remove galaxies too, we could call it `minus.Q13442814-Q318`, so the files would be `minus.Q13442814-Q318.part.quantity.tsv` (the idea of `all` is in contrast to `minus`). We can also have files that start with Qnodes, e.g, `Q5.part.quantity.tsv`; constructing such files is harder as we don't want dangling nodes in the item file.

In [3]:
os.environ['WIKIDATA_HOME'] = home
os.environ['OUT'] = "{}/{}".format(os.environ['WIKIDATA_HOME'], output_folder)
os.environ['TEMP'] = "{}/{}".format(os.environ['WIKIDATA_HOME'], temp_folder)
os.environ['kgtk'] = "kgtk"
os.environ['kgtk'] = "time kgtk --debug"
wikidata_home = os.environ['WIKIDATA_HOME']

In [4]:
!echo $WIKIDATA_HOME
!echo $OUT
!echo $TEMP
!echo $kgtk

/Users/pedroszekely/Downloads/kypher
/Users/pedroszekely/Downloads/kypher/output.all.10
/Users/pedroszekely/Downloads/kypher/temp.all.10
time kgtk --debug


In [5]:
cd $wikidata_home

/Users/pedroszekely/Downloads/kypher


In [6]:
!mkdir $OUT
!mkdir $TEMP

mkdir: /Users/pedroszekely/Downloads/kypher/output.all.10: File exists


Clean up the output and temp folders before we start

In [7]:
!rm $OUT/*.tsv $OUT/*.tsv.gz
!rm $TEMP/*.tsv $TEMP/*.tsv.gz

rm: /Users/pedroszekely/Downloads/kypher/temp.all.10/*.tsv: No such file or directory
rm: /Users/pedroszekely/Downloads/kypher/temp.all.10/*.tsv.gz: No such file or directory


In [8]:
if delete_database:
    print("Deleting database")
    !rm $TEMP/wikidata.sqlite3.db

Deleting database
rm: /Users/pedroszekely/Downloads/kypher/temp.all.10/wikidata.sqlite3.db: No such file or directory


The `all` file contains 100M edges of the full dump, `all.10` contains 10M edges. This is for testing, as we should run on the full edges file.

In [9]:
%env STORE=$wikidata_home/temp/wikidata.sqlite3.db
# %env EDGES=$wikidata_home/all.10.tsv
%env EDGES=$wikidata_home/$wiki_file

#%env QUALS=$wikidata_home/wikidata-20200803-all-qualifiers.tsv.gz
#%env LABELS=$wikidata_home/wikidata-20200803-all-labels-en-sorted.tsv.gz

env: STORE=/Users/pedroszekely/Downloads/kypher/temp/wikidata.sqlite3.db
env: EDGES=/Users/pedroszekely/Downloads/kypher/all.10.tsv.gz


Uncomment the line below to remove the sqllite2 database. It takes a long time to load all the data and create indices, so don't remove the database unless you change files that have already been loaded and you need to force a reload.

### Get a sample and force importing the edge file into the database

In [10]:
!$kgtk query -i $EDGES --limit 10 --graph-cache $STORE

[2020-09-30 11:21:25 query]: SQL Translation:
---------------------------------------------
  SELECT *
     FROM graph_8 AS graph_8_c1
     LIMIT ?
  PARAS: [10]
---------------------------------------------
id	node1	label	node2	rank	node2;magnitude	node2;unit	node2;date	node2;item	node2;lower	node2;upper	node2;latitude	node2;longitude	node2;precision	node2;calendar	node2;entity-type	node2;wikidatatype
Q45-label-en	Q45	label	'Portugal'@en													
Q45-label-fr	Q45	label	'Portugal'@fr													
Q45-label-nb	Q45	label	'Portugal'@nb													
Q45-label-it	Q45	label	'Portogallo'@it													
Q45-label-ru	Q45	label	'Португалия'@ru													
Q45-label-nl	Q45	label	'Portugal'@nl													
Q45-label-es	Q45	label	'Portugal'@es													
Q45-label-de	Q45	label	'Portugal'@de													
Q45-label-pl	Q45	label	'Portugalia'@pl													
Q45-label-be-tarask	Q45	label	'Партугалія'@be-tarask													
        2.22 real         0.60 user         0.16 sys


Force creation of the index on the label column

In [11]:
!$kgtk query -i $EDGES --graph-cache $STORE -o - \
    --match '(i)-[:P31]->(c)' \
    --limit 5

[2020-09-30 11:21:26 query]: SQL Translation:
---------------------------------------------
  SELECT *
     FROM graph_8 AS graph_8_c1
     WHERE graph_8_c1."label"=?
     LIMIT ?
  PARAS: ['P31', 5]
---------------------------------------------
id	node1	label	node2	rank	node2;magnitude	node2;unit	node2;date	node2;item	node2;lower	node2;upper	node2;latitude	node2;longitude	node2;precision	node2;calendar	node2;entity-type	node2;wikidatatype
Q45-P31-1	Q45	P31	Q3624078	normal				Q3624078							item	wikibase-item
Q45-P31-2	Q45	P31	Q6256	normal				Q6256							item	wikibase-item
Q45-P31-3	Q45	P31	Q20181813	normal				Q20181813							item	wikibase-item
Q140-P31-1	Q140	P31	Q16521	normal				Q16521							item	wikibase-item
Q183-P31-1	Q183	P31	Q3624078	preferred				Q3624078							item	wikibase-item
        0.71 real         0.57 user         0.13 sys


Force creation of the index on the node2 column

In [12]:
!$kgtk query -i $EDGES --graph-cache $STORE -o - \
    --match '(i)-[r]->(:Q5)' \
    --limit 5

[2020-09-30 11:21:27 query]: SQL Translation:
---------------------------------------------
  SELECT *
     FROM graph_8 AS graph_8_c1
     WHERE graph_8_c1."node2"=?
     LIMIT ?
  PARAS: ['Q5', 5]
---------------------------------------------
id	node1	label	node2	rank	node2;magnitude	node2;unit	node2;date	node2;item	node2;lower	node2;upper	node2;latitude	node2;longitude	node2;precision	node2;calendar	node2;entity-type	node2;wikidatatype
Q1253-P31-1	Q1253	P31	Q5	normal				Q5							item	wikibase-item
Q1526-P31-1	Q1526	P31	Q5	normal				Q5							item	wikibase-item
Q3794-P31-1	Q3794	P31	Q5	normal				Q5							item	wikibase-item
Q4291-P31-1	Q4291	P31	Q5	normal				Q5							item	wikibase-item
Q4489-P31-1	Q4489	P31	Q5	normal				Q5							item	wikibase-item
        0.70 real         0.56 user         0.12 sys


### Count the number of edges

In [13]:
!$kgtk query -i $EDGES --graph-cache $STORE \
    --match 'all: ()-[r]->()' \
    --return 'count(r) as count' \
    --limit 10

[2020-09-30 11:21:28 query]: SQL Translation:
---------------------------------------------
  SELECT count(graph_8_c1."id") "count"
     FROM graph_8 AS graph_8_c1
     LIMIT ?
  PARAS: [10]
---------------------------------------------
count
9999999
        6.80 real         1.57 user         1.54 sys


### Get the distribution of the label column
I would like to have it sorted numerically, but don't know how to make it happen

In [14]:
!$kgtk unique --column label -i $EDGES / sort2 -c node2 -r -o $OUT/all-distribution.tsv 

       43.44 real        42.83 user         0.79 sys


In [15]:
!head $OUT/all-distribution.tsv | column -t -s $'\t' 

node1  label  node2
P3987  count  998
P410   count  987
P575   count  985
P6879  count  98
P6562  count  98
P5395  count  98
P3153  count  98
P4933  count  97
P3135  count  97


### Compute files with labels, aliases and descriptions
Return the id, node1, label and node2 columns

In [16]:
!$kgtk query -i $EDGES --graph-cache $STORE -o $OUT/part.label.tsv.gz \
    --match '(n1)-[l:label]->(n2)' \
    --return 'l, n1, l.label, n2' 

[2020-09-30 11:22:19 query]: SQL Translation:
---------------------------------------------
  SELECT graph_8_c1."id", graph_8_c1."node1", graph_8_c1."label", graph_8_c1."node2"
     FROM graph_8 AS graph_8_c1
     WHERE graph_8_c1."label"=?
  PARAS: ['label']
---------------------------------------------
       20.17 real        17.45 user         1.26 sys


In [17]:
!$kgtk query -i $EDGES --graph-cache $STORE -o $OUT/part.alias.tsv.gz \
    --match '(n1)-[l:alias]->(n2)' \
    --return 'l, n1, l.label, n2'

[2020-09-30 11:22:39 query]: SQL Translation:
---------------------------------------------
  SELECT graph_8_c1."id", graph_8_c1."node1", graph_8_c1."label", graph_8_c1."node2"
     FROM graph_8 AS graph_8_c1
     WHERE graph_8_c1."label"=?
  PARAS: ['alias']
---------------------------------------------
        3.65 real         2.86 user         0.42 sys


In [18]:
!$kgtk query -i $EDGES --graph-cache $STORE -o $OUT/part.description.tsv.gz \
    --match '(n1)-[l:description]->(n2)' \
    --return 'l, n1, l.label, n2'

[2020-09-30 11:22:43 query]: SQL Translation:
---------------------------------------------
  SELECT graph_8_c1."id", graph_8_c1."node1", graph_8_c1."label", graph_8_c1."node2"
     FROM graph_8 AS graph_8_c1
     WHERE graph_8_c1."label"=?
  PARAS: ['description']
---------------------------------------------
       60.09 real        50.85 user         3.29 sys


### Now create files with the English labels, aliases and descriptions

In [19]:
!$kgtk query -i $OUT/part.label.tsv.gz --graph-cache $STORE -o $OUT/part.label.en.tsv.gz \
    --match '()-[]->(n2)' \
    --where 'n2.kgtk_lqstring_lang_suffix = "en"' 

[2020-09-30 11:23:43 sqlstore]: DROP graph data table graph_18 from /Users/pedroszekely/Downloads/kypher/output.all.10/part.label.tsv.gz
[2020-09-30 11:23:44 sqlstore]: IMPORT graph directly into table graph_38 from /Users/pedroszekely/Downloads/kypher/output.all.10/part.label.tsv.gz ...
[2020-09-30 11:23:54 query]: SQL Translation:
---------------------------------------------
  SELECT *
     FROM graph_38 AS graph_38_c1
     WHERE (kgtk_lqstring_lang_suffix(graph_38_c1."node2") = ?)
  PARAS: ['en']
---------------------------------------------
       15.08 real        18.64 user         1.55 sys


In [20]:
!$kgtk query -i $OUT/part.alias.tsv.gz --graph-cache $STORE -o $OUT/part.alias.en.tsv.gz \
    --match '()-[]->(n2)' \
    --where 'n2.kgtk_lqstring_lang_suffix = "en"'

[2020-09-30 11:23:58 sqlstore]: DROP graph data table graph_19 from /Users/pedroszekely/Downloads/kypher/output.all.10/part.alias.tsv.gz
[2020-09-30 11:23:58 sqlstore]: IMPORT graph directly into table graph_39 from /Users/pedroszekely/Downloads/kypher/output.all.10/part.alias.tsv.gz ...
[2020-09-30 11:23:59 query]: SQL Translation:
---------------------------------------------
  SELECT *
     FROM graph_39 AS graph_39_c1
     WHERE (kgtk_lqstring_lang_suffix(graph_39_c1."node2") = ?)
  PARAS: ['en']
---------------------------------------------
        2.66 real         3.06 user         0.29 sys


In [21]:
!$kgtk query -i $OUT/part.description.tsv.gz --graph-cache $STORE -o $OUT/part.description.en.tsv.gz \
    --match '()-[]->(n2)' \
    --where 'n2.kgtk_lqstring_lang_suffix = "en"' 

[2020-09-30 11:24:01 sqlstore]: DROP graph data table graph_20 from /Users/pedroszekely/Downloads/kypher/output.all.10/part.description.tsv.gz
[2020-09-30 11:24:06 sqlstore]: IMPORT graph directly into table graph_40 from /Users/pedroszekely/Downloads/kypher/output.all.10/part.description.tsv.gz ...
[2020-09-30 11:24:45 query]: SQL Translation:
---------------------------------------------
  SELECT *
     FROM graph_40 AS graph_40_c1
     WHERE (kgtk_lqstring_lang_suffix(graph_40_c1."node2") = ?)
  PARAS: ['en']
---------------------------------------------
       53.85 real        71.20 user         5.32 sys


Let's sample these files to see what they look like:

* we are getting all variants of English, we really want `en` only
* the labels have the language tags, how do we output only the string without the language tag?

In [51]:
!gzcat $OUT/part.label.en.tsv.gz | head | column -t -s $'\t' 

id             node1  label  node2
Q45-label-en   Q45    label  'Portugal'@en
Q140-label-en  Q140   label  'lion'@en
Q183-label-en  Q183   label  'Germany'@en
Q317-label-en  Q317   label  'dictatorship'@en
Q433-label-en  Q433   label  'Gmina Kurów'@en
Q514-label-en  Q514   label  'anatomy'@en
Q595-label-en  Q595   label  'The Intouchables'@en
Q647-label-en  Q647   label  'Rennes'@en
Q716-label-en  Q716   label  'titanium'@en
gzcat: error writing to output: Broken pipe
gzcat: /Users/pedroszekely/Downloads/kypher/output.all.10/part.label.en.tsv.gz: uncompress failed


### Compute the distribution of the number of edges for each Wikidata type

In [23]:
!$kgtk unique --column 'node2;wikidatatype' -i $EDGES / sort2 -c node2 -r | gzip > $OUT/all.wikidatatype.distribution.tsv.gz

       42.94 real        42.38 user         0.81 sys


In [24]:
!gzcat $OUT/all.wikidatatype.distribution.tsv.gz | column -t -s $'\t' 

node1              label  node2
time               count  76936
wikibase-item      count  729535
math               count  70
wikibase-form      count  7
quantity           count  69823
string             count  68283
external-id        count  416408
commonsMedia       count  36794
globe-coordinate   count  26063
monolingualtext    count  24131
musical-notation   count  2
geo-shape          count  183
wikibase-property  count  148
url                count  12874


### Create a file to contain the edges for each wikidata type

In [25]:
types = [
    "time",
    "wikibase-item",
    "math",
    "wikibase-form",
    "quantity",
    "string",
    "external-id",
    "commonsMedia",
    "globe-coordinate",
    "monolingualtext",
    "musical-notation",
    "geo-shape",
    "wikibase-property",
    "url",
]

command = "$kgtk query -i $EDGES --graph-cache $STORE -o $OUT/part.TYPE_FILE.tsv.gz \
    --match '(n1)-[l]->(n2 {wikidatatype: type})' \
    --return 'l, n1, l.label, n2'\
    --where 'type = \"TYPE\"'"
for type in types:
    cmd = command.replace("TYPE_FILE", type)
    cmd = cmd.replace("TYPE", type)

    print(cmd)
    os.system(cmd)

$kgtk query -i $EDGES --graph-cache $STORE -o $OUT/part.time.tsv.gz     --match '(n1)-[l]->(n2 {wikidatatype: type})'     --return 'l, n1, l.label, n2'    --where 'type = "time"'
$kgtk query -i $EDGES --graph-cache $STORE -o $OUT/part.wikibase-item.tsv.gz     --match '(n1)-[l]->(n2 {wikidatatype: type})'     --return 'l, n1, l.label, n2'    --where 'type = "wikibase-item"'
$kgtk query -i $EDGES --graph-cache $STORE -o $OUT/part.math.tsv.gz     --match '(n1)-[l]->(n2 {wikidatatype: type})'     --return 'l, n1, l.label, n2'    --where 'type = "math"'
$kgtk query -i $EDGES --graph-cache $STORE -o $OUT/part.wikibase-form.tsv.gz     --match '(n1)-[l]->(n2 {wikidatatype: type})'     --return 'l, n1, l.label, n2'    --where 'type = "wikibase-form"'
$kgtk query -i $EDGES --graph-cache $STORE -o $OUT/part.quantity.tsv.gz     --match '(n1)-[l]->(n2 {wikidatatype: type})'     --return 'l, n1, l.label, n2'    --where 'type = "quantity"'
$kgtk query -i $EDGES --graph-cache $STORE -o $OUT/part.strin

### Create a file with the sitelinks

In [26]:
!$kgtk query -i $EDGES --graph-cache $STORE -o $OUT/part.wikipedia_sitelink.tsv.gz \
    --match '(n1)-[l:wikipedia_sitelink]->(n2)' \
    --return 'l, n1, l.label, n2' 

[2020-09-30 11:26:37 query]: SQL Translation:
---------------------------------------------
  SELECT graph_8_c1."id", graph_8_c1."node1", graph_8_c1."label", graph_8_c1."node2"
     FROM graph_8 AS graph_8_c1
     WHERE graph_8_c1."label"=?
  PARAS: ['wikipedia_sitelink']
---------------------------------------------
        8.27 real         6.88 user         0.71 sys


### Create a file that specifies for each node whether it is an item or a property

In [27]:
!$kgtk query -i $EDGES --graph-cache $STORE -o $OUT/part.type.tsv.gz \
    --match '(n1)-[l:type]->(n2)' \
    --return 'l, n1, l.label, n2' 

[2020-09-30 11:26:46 query]: SQL Translation:
---------------------------------------------
  SELECT graph_8_c1."id", graph_8_c1."node1", graph_8_c1."label", graph_8_c1."node2"
     FROM graph_8 AS graph_8_c1
     WHERE graph_8_c1."label"=?
  PARAS: ['type']
---------------------------------------------
        3.21 real         2.50 user         0.58 sys


### Create the P31 and P279 files

In [28]:
!$kgtk query -i $EDGES --graph-cache $STORE -o $OUT/all.P31.tsv.gz \
    --match '(n1)-[l:P31]->(n2)' \
    --return 'l, n1, l.label, n2' 

[2020-09-30 11:26:49 query]: SQL Translation:
---------------------------------------------
  SELECT graph_8_c1."id", graph_8_c1."node1", graph_8_c1."label", graph_8_c1."node2"
     FROM graph_8 AS graph_8_c1
     WHERE graph_8_c1."label"=?
  PARAS: ['P31']
---------------------------------------------
        2.73 real         2.17 user         0.45 sys


In [29]:
!$kgtk query -i $EDGES --graph-cache $STORE -o $OUT/all.P279.tsv.gz \
    --match '(n1)-[l:P279]->(n2)' \
    --return 'l, n1, l.label, n2' 

[2020-09-30 11:26:52 query]: SQL Translation:
---------------------------------------------
  SELECT graph_8_c1."id", graph_8_c1."node1", graph_8_c1."label", graph_8_c1."node2"
     FROM graph_8 AS graph_8_c1
     WHERE graph_8_c1."label"=?
  PARAS: ['P279']
---------------------------------------------
        0.79 real         0.64 user         0.14 sys


In [30]:
!gzcat $OUT/all.P31.tsv.gz | head | column -t -s $'\t' 

gzcat: error writing to output: Broken pipe
gzcat: /Users/pedroszekely/Downloads/kypher/output.all.10/all.P31.tsv.gz: uncompress failed
id          node1  label  node2
Q45-P31-1   Q45    P31    Q3624078
Q45-P31-2   Q45    P31    Q6256
Q45-P31-3   Q45    P31    Q20181813
Q140-P31-1  Q140   P31    Q16521
Q183-P31-1  Q183   P31    Q3624078
Q183-P31-2  Q183   P31    Q43702
Q183-P31-3  Q183   P31    Q7270
Q183-P31-4  Q183   P31    Q619610
Q183-P31-5  Q183   P31    Q4209223


In [31]:
!$kgtk cat -i $OUT/all.P279.tsv.gz -i $OUT/all.P31.tsv.gz -o $OUT/all.P31_P279.tsv.gz 

        2.57 real         2.37 user         0.14 sys


In [32]:
!gzcat $OUT/all.P31_P279.tsv | head | column -t -s $'\t' 

id            node1  label  node2
Q317-P279-1   Q317   P279   Q173424
Q514-P279-1   Q514   P279   Q420
Q514-P279-2   Q514   P279   Q11190
Q716-P279-1   Q716   P279   Q19588
Q716-P279-2   Q716   P279   Q428766
Q901-P279-1   Q901   P279   Q1650915
Q901-P279-2   Q901   P279   Q20826540
Q1071-P279-1  Q1071  P279   Q34749
Q1071-P279-2  Q1071  P279   Q8008
gzcat: error writing to output: Broken pipe
gzcat: /Users/pedroszekely/Downloads/kypher/output.all.10/all.P31_P279.tsv.gz: uncompress failed


### Create the file that contains all nodes reachable via P279 starting from a node2 in P31 or a node1 in P279

First compute the roots

In [33]:
!$kgtk query -i $OUT/all.P279.tsv.gz --graph-cache $STORE -o $TEMP/P279.n1.tsv.gz \
    --match '(n1)-[]->()' \
    --return 'n1 as node' 

[2020-09-30 11:26:56 sqlstore]: DROP graph data table graph_21 from /Users/pedroszekely/Downloads/kypher/output.all.10/all.P279.tsv.gz
[2020-09-30 11:26:56 sqlstore]: IMPORT graph directly into table graph_41 from /Users/pedroszekely/Downloads/kypher/output.all.10/all.P279.tsv.gz ...
[2020-09-30 11:26:56 query]: SQL Translation:
---------------------------------------------
  SELECT graph_41_c1."node1" "node"
     FROM graph_41 AS graph_41_c1
  PARAS: []
---------------------------------------------
        0.75 real         0.59 user         0.15 sys


In [34]:
!$kgtk query -i $OUT/all.P31.tsv.gz --graph-cache $STORE  -o $TEMP/P31.n2.tsv.gz \
    --match '()-[]->(n2)' \
    --return 'n2 as node' 

[2020-09-30 11:26:57 sqlstore]: DROP graph data table graph_22 from /Users/pedroszekely/Downloads/kypher/output.all.10/all.P31.tsv.gz
[2020-09-30 11:26:57 sqlstore]: IMPORT graph directly into table graph_42 from /Users/pedroszekely/Downloads/kypher/output.all.10/all.P31.tsv.gz ...
[2020-09-30 11:26:57 query]: SQL Translation:
---------------------------------------------
  SELECT graph_42_c1."node2" "node"
     FROM graph_42 AS graph_42_c1
  PARAS: []
---------------------------------------------
        1.40 real         1.37 user         0.22 sys


In [35]:
!$kgtk cat --mode NONE -i $TEMP/P31.n2.tsv.gz $TEMP/P279.n1.tsv.gz \
    / compact --mode NONE --columns node \
    > $TEMP/P279.roots.tsv

        2.34 real         2.95 user         0.44 sys


Now we can invoke the reachable-nodes command

In [36]:
!$kgtk reachable-nodes \
    --rootfile $TEMP/P279.roots.tsv \
    --rootfilecolumn 0 \
    --subj 1 --pred 2 --obj 3 \
    -i $OUT/all.P279.tsv.gz \
    | kgtk sort2 \
    | gzip > $TEMP/P279.reachable.tsv.gz

        2.19 real         0.86 user         0.21 sys


The reachable-nodes command produces edges labeled `reachable`, so we need one command to rename them.

In [37]:
!$kgtk query -i $TEMP/P279.reachable.tsv.gz --graph-cache $STORE  -o $TEMP/P279star.1.tsv.gz \
    --match '(n1)-[]->(n2)' \
    --return 'n1, "P279star" as label, n2 as node2' 

[2020-09-30 11:27:03 sqlstore]: DROP graph data table graph_23 from /Users/pedroszekely/Downloads/kypher/temp.all.10/P279.reachable.tsv.gz
[2020-09-30 11:27:03 sqlstore]: IMPORT graph directly into table graph_43 from /Users/pedroszekely/Downloads/kypher/temp.all.10/P279.reachable.tsv.gz ...
[2020-09-30 11:27:03 query]: SQL Translation:
---------------------------------------------
  SELECT graph_43_c1."node1", ? "label", graph_43_c1."node2" "node2"
     FROM graph_43 AS graph_43_c1
  PARAS: ['P279star']
---------------------------------------------
        0.84 real         0.66 user         0.16 sys


We also want `P279star` to be relflexive, ie, contain `(n1)-[:P279star]->(n1)` for all node1

In [38]:
!$kgtk query -i $TEMP/P279.reachable.tsv.gz --graph-cache $STORE  -o $TEMP/P279star.2.tsv.gz \
    --match '(n1)-[]->(n2)' \
    --return 'n1 as node1, "P279star" as label, n1 as node2' 

[2020-09-30 11:27:04 query]: SQL Translation:
---------------------------------------------
  SELECT graph_43_c1."node1" "node1", ? "label", graph_43_c1."node1" "node2"
     FROM graph_43 AS graph_43_c1
  PARAS: ['P279star']
---------------------------------------------
        0.81 real         0.65 user         0.14 sys


In [39]:
!$kgtk query -i $TEMP/P279.reachable.tsv.gz --graph-cache $STORE  -o $TEMP/P279star.3.tsv.gz \
    --match '(n1)-[]->(n2)' \
    --return 'n2 as node1, "P279star" as label, n2 as node2' 

[2020-09-30 11:27:05 query]: SQL Translation:
---------------------------------------------
  SELECT graph_43_c1."node2" "node1", ? "label", graph_43_c1."node2" "node2"
     FROM graph_43 AS graph_43_c1
  PARAS: ['P279star']
---------------------------------------------
        0.79 real         0.64 user         0.13 sys


In [40]:
!$kgtk query -i $OUT/all.P31.tsv.gz --graph-cache $STORE  -o $TEMP/P279star.4.tsv.gz \
    --match '(n1)-[]->(n2)' \
    --return 'n2 as node1, "P279star" as label, n2 as node2' 

[2020-09-30 11:27:06 query]: SQL Translation:
---------------------------------------------
  SELECT graph_42_c1."node2" "node1", ? "label", graph_42_c1."node2" "node2"
     FROM graph_42 AS graph_42_c1
  PARAS: ['P279star']
---------------------------------------------
        1.30 real         1.11 user         0.16 sys


Now we can concatenate these files to produce the final output

In [41]:
!$kgtk cat --mode NONE -i $TEMP/P279star.1.tsv.gz $TEMP/P279star.2.tsv.gz $TEMP/P279star.3.tsv.gz $TEMP/P279star.4.tsv.gz \
    | kgtk compact \
    | kgtk sort2 \
    | kgtk add-id --id-style node1-label-node2-num \
    | gzip > $OUT/all.P279star.tsv.gz

        1.78 real         1.50 user         0.18 sys


This is difficult to test with our Wikidata subset because our hierarchy is very sparse.

This is how we would do the typical `?item P31/P279* ?class` in Kypher. 
The example shows how to get all the `n1` that are instances of subclasses of beer (q44).

In [42]:
!$kgtk query -i $OUT/all.P31.tsv.gz -i $OUT/all.P279star.tsv.gz --graph-cache $STORE  -o - \
    --match 'P31: (n1)-[:P31]->(c), P279star: (c)-[]->(:Q44)' \
    --return 'count(n1) as count'

[2020-09-30 11:27:10 sqlstore]: DROP graph data table graph_24 from /Users/pedroszekely/Downloads/kypher/output.all.10/all.P279star.tsv.gz
[2020-09-30 11:27:10 sqlstore]: IMPORT graph directly into table graph_44 from /Users/pedroszekely/Downloads/kypher/output.all.10/all.P279star.tsv.gz ...
[2020-09-30 11:27:10 query]: SQL Translation:
---------------------------------------------
  SELECT count(graph_42_c1."node1") "count"
     FROM graph_42 AS graph_42_c1, graph_44 AS graph_44_c2
     WHERE graph_42_c1."label"=?
     AND graph_44_c2."node2"=?
     AND graph_42_c1."node2"=graph_44_c2."node1"
  PARAS: ['P31', 'Q44']
---------------------------------------------
[2020-09-30 11:27:10 sqlstore]: CREATE INDEX on table graph_42 column node2 ...
[2020-09-30 11:27:11 sqlstore]: ANALYZE INDEX on table graph_42 column node2 ...
[2020-09-30 11:27:11 sqlstore]: CREATE INDEX on table graph_44 column node2 ...
[2020-09-30 11:27:11 sqlstore]: ANALYZE INDEX on table graph_44 column node2 ...
[2020-0

### Create a file to do generalized Is-A queries
The idea is that `(n1)-[:isa]->(n2)` when `(n1)-[:P31]->(n2)` or `(n1)-[:P279]->(n2)`

We do this by concatenating the files and renaming the relation

In [43]:
!$kgtk cat -i $OUT/all.P31.tsv.gz $OUT/all.P279.tsv.gz \
    | gzip > $TEMP/isa.1.tsv.gz

        1.43 real         1.26 user         0.14 sys


In [44]:
!$kgtk query -i $TEMP/isa.1.tsv.gz --graph-cache $STORE  -o $OUT/all.isa.tsv.gz \
    --match '(n1)-[]->(n2)' \
    --return 'n1, "isa" as label, n2' 

[2020-09-30 11:27:13 sqlstore]: DROP graph data table graph_25 from /Users/pedroszekely/Downloads/kypher/temp.all.10/isa.1.tsv.gz
[2020-09-30 11:27:13 sqlstore]: IMPORT graph directly into table graph_45 from /Users/pedroszekely/Downloads/kypher/temp.all.10/isa.1.tsv.gz ...
[2020-09-30 11:27:14 query]: SQL Translation:
---------------------------------------------
  SELECT graph_45_c1."node1", ? "label", graph_45_c1."node2"
     FROM graph_45 AS graph_45_c1
  PARAS: ['isa']
---------------------------------------------
        2.10 real         1.91 user         0.25 sys


Example of how to use the `isa` relation

In [45]:
!$kgtk query -i $OUT/all.isa.tsv.gz -i $OUT/all.P279star.tsv.gz --graph-cache $STORE  -o - \
    --match 'isa: (n1)-[l:isa]->(c), P279star: (c)-[]->(:Q44)' \
    --return 'distinct n1, l.label, "Q44" as node2' \
    --limit 10

[2020-09-30 11:27:15 sqlstore]: DROP graph data table graph_28 from /Users/pedroszekely/Downloads/kypher/output.all.10/all.isa.tsv.gz
[2020-09-30 11:27:15 sqlstore]: IMPORT graph directly into table graph_28 from /Users/pedroszekely/Downloads/kypher/output.all.10/all.isa.tsv.gz ...
[2020-09-30 11:27:16 query]: SQL Translation:
---------------------------------------------
  SELECT DISTINCT graph_28_c1."node1", graph_28_c1."label", ? "node2"
     FROM graph_28 AS graph_28_c1, graph_44 AS graph_44_c2
     WHERE graph_28_c1."label"=?
     AND graph_44_c2."node2"=?
     AND graph_28_c1."node2"=graph_44_c2."node1"
     LIMIT ?
  PARAS: ['Q44', 'isa', 'Q44', 10]
---------------------------------------------
[2020-09-30 11:27:16 sqlstore]: CREATE INDEX on table graph_28 column label ...
[2020-09-30 11:27:16 sqlstore]: ANALYZE INDEX on table graph_28 column label ...
[2020-09-30 11:27:16 sqlstore]: CREATE INDEX on table graph_28 column node2 ...
[2020-09-30 11:27:16 sqlstore]: ANALYZE INDEX on

### Creating a subset of Wikidata without scholarly articles (Q13442814)
First create a file with the schloarly articles

In [46]:
!$kgtk query -i $OUT/all.isa.tsv.gz -i $OUT/all.P279star.tsv.gz --graph-cache $STORE  -o $OUT/all.isa.Q13442814.tsv.gz \
    --match 'isa: (n1)-[l:isa]->(n2:Q13442814)' \
    --return 'distinct n1, l.label, n2'

[2020-09-30 11:27:17 query]: SQL Translation:
---------------------------------------------
  SELECT DISTINCT graph_28_c1."node1", graph_28_c1."label", graph_28_c1."node2"
     FROM graph_28 AS graph_28_c1
     WHERE graph_28_c1."label"=?
     AND graph_28_c1."node2"=?
  PARAS: ['isa', 'Q13442814']
---------------------------------------------
        0.84 real         0.67 user         0.15 sys


Now we need to remove from `$EDGES` any edge where node1 or node2 is in node1 of `$OUT/all.isa.Q13442814.tsv`. The result will be `$OUT/minus.Q13442814.tsv`. We can then run the whole notebook with this new file as $EDGES and compute all the product files in a new output directory

In [47]:
!gzcat $OUT/all.isa.Q13442814.tsv | head | column -t -s $'\t' 

node1     label  node2
Q1801903  isa    Q13442814


## Summary

In [48]:
!wc -l $OUT/*.tsv $OUT/*.tsv.gz $EDGES

    4882 /Users/pedroszekely/Downloads/kypher/output.all.10/all-distribution.tsv
     143 /Users/pedroszekely/Downloads/kypher/output.all.10/all.P279.tsv.gz
     793 /Users/pedroszekely/Downloads/kypher/output.all.10/all.P279star.tsv.gz
    5348 /Users/pedroszekely/Downloads/kypher/output.all.10/all.P31.tsv.gz
    5512 /Users/pedroszekely/Downloads/kypher/output.all.10/all.P31_P279.tsv.gz
       0 /Users/pedroszekely/Downloads/kypher/output.all.10/all.isa.Q13442814.tsv.gz
    2111 /Users/pedroszekely/Downloads/kypher/output.all.10/all.isa.tsv.gz
       3 /Users/pedroszekely/Downloads/kypher/output.all.10/all.wikidatatype.distribution.tsv.gz
    1814 /Users/pedroszekely/Downloads/kypher/output.all.10/part.alias.en.tsv.gz
   13162 /Users/pedroszekely/Downloads/kypher/output.all.10/part.alias.tsv.gz
    3125 /Users/pedroszekely/Downloads/kypher/output.all.10/part.commonsMedia.tsv.gz
    6171 /Users/pedroszekely/Downloads/kypher/output.all.10/part.description.en.tsv.gz
  227442 /Users/pedr

Number of distinct items in our dataset

In [49]:
!$kgtk query -i $EDGES --graph-cache $STORE  -o - \
    --match '(n1)-[]->()' \
    --return 'count(distinct n1) as count'

[2020-09-30 11:27:18 query]: SQL Translation:
---------------------------------------------
  SELECT count(DISTINCT graph_8_c1."node1") "count"
     FROM graph_8 AS graph_8_c1
  PARAS: []
---------------------------------------------
count
156559
        5.91 real         4.82 user         1.00 sys
